In [1]:
# Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from ast import literal_eval
import ast
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
# Importing files
ratings=pd.read_csv('data/train.csv', low_memory=False)
links=pd.read_csv('data/links.csv', low_memory=False)
movies=pd.read_csv('data/movies.csv', low_memory=False)
tags=pd.read_csv('data/tags.csv', low_memory=False)
genome_scores=pd.read_csv('data/genome_scores.csv', low_memory=False)
genome_tags=pd.read_csv('data/genome_tags.csv', low_memory=False)
test=pd.read_csv('data/test.csv', low_memory=False)

# Styles to view dataframes side by side
from IPython.core.display import HTML
def multi_table(table_list):
    ''' Acceps a list of IpyTable objects and returns a table which contains each IpyTable in a cell
    '''
    return HTML(
        '<table><tr style="background-color:white;">' + 
        ''.join(['<td>' + table._repr_html_() + '</td>' for table in table_list]) +
        '</tr></table>'
    )

# Sample of our Dataset
display(multi_table([ratings.head(10),movies.head(10),tags.head(10),genome_tags.head(10),genome_scores.head(10),links.head(10),test.head(10)]))


,userId,movieId,rating,timestamp
0,5163,57669,4.0,1518349992
1,106343,5,4.5,1206238739
2,146790,5459,5.0,1076215539
3,106362,32296,2.0,1423042565
4,9041,366,3.0,833375837
5,120949,81768,3.0,1289595242
6,19630,62049,4.0,1246729817
7,21066,2282,1.0,945785907
8,117563,120474,4.0,1515108225
9,144018,1997,5.0,1109967647


In [3]:
imdb_data=pd.read_csv('data/imdb_data.csv', low_memory=False)
display(imdb_data.head(10))

,movieId,title_cast,director,runtime,budget,plot_keywords
0,1,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,John Lasseter,81.0,"$30,000,000",toy|rivalry|cowboy|cgi animation
1,2,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,Jonathan Hensleigh,104.0,"$65,000,000",board game|adventurer|fight|game
2,3,Walter Matthau|Jack Lemmon|Sophia Loren|Ann-Ma...,Mark Steven Johnson,101.0,"$25,000,000",boat|lake|neighbor|rivalry
3,4,Whitney Houston|Angela Bassett|Loretta Devine|...,Terry McMillan,124.0,"$16,000,000",black american|husband wife relationship|betra...
4,5,Steve Martin|Diane Keaton|Martin Short|Kimberl...,Albert Hackett,106.0,"$30,000,000",fatherhood|doberman|dog|mansion
5,6,Al Pacino|Robert De Niro|Val Kilmer|Jon Voight...,Michael Mann,170.0,"$60,000,000",gun battle|organized crime|honor|criminal inve...
6,7,Harrison Ford|Julia Ormond|Greg Kinnear|Nancy ...,Samuel A. Taylor,127.0,"$58,000,000",character name as title|chauffeur|estate|playboy
7,8,Jonathan Taylor Thomas|Brad Renfro|Eric Schwei...,Mark Twain,97.0,NaN,rescue from drowning|near drowning|character n...
8,9,Jean-Claude Van Damme|Powers Boothe|Raymond J....,Karen Elise Baldwin,111.0,"$35,000,000",hockey|death in title|two word title|terrorist
9,10,Pierce Brosnan|Sean Bean|Izabella Scorupco|Fam...,Ian Fleming,130.0,"$60,000,000",official james bond series|james bond characte...


In [4]:
new_imdb_data = imdb_data.drop(['movieId'],axis = 1)
movies_metadata=pd.concat([movies, new_imdb_data.reindex(movies.index)],axis = 1)
display(movies_metadata.head())

,movieId,title,genres,title_cast,director,runtime,budget,plot_keywords
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,John Lasseter,81.0,"$30,000,000",toy|rivalry|cowboy|cgi animation
1,2,Jumanji (1995),Adventure|Children|Fantasy,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,Jonathan Hensleigh,104.0,"$65,000,000",board game|adventurer|fight|game
2,3,Grumpier Old Men (1995),Comedy|Romance,Walter Matthau|Jack Lemmon|Sophia Loren|Ann-Ma...,Mark Steven Johnson,101.0,"$25,000,000",boat|lake|neighbor|rivalry
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Whitney Houston|Angela Bassett|Loretta Devine|...,Terry McMillan,124.0,"$16,000,000",black american|husband wife relationship|betra...
4,5,Father of the Bride Part II (1995),Comedy,Steve Martin|Diane Keaton|Martin Short|Kimberl...,Albert Hackett,106.0,"$30,000,000",fatherhood|doberman|dog|mansion


In [5]:
movies_metadata.info()

movies_metadata[movies_metadata['title'].isnull() ]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movieId        62423 non-null  int64  
 1   title          62423 non-null  object 
 2   genres         62423 non-null  object 
 3   title_cast     17210 non-null  object 
 4   director       17404 non-null  object 
 5   runtime        15189 non-null  float64
 6   budget         7906 non-null   object 
 7   plot_keywords  16200 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 3.8+ MB


,movieId,title,genres,title_cast,director,runtime,budget,plot_keywords


In [6]:
# Remove null budget from original titles
movies_metadata = movies_metadata[pd.notnull(movies_metadata['title'])]

movies_metadata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62423 entries, 0 to 62422
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movieId        62423 non-null  int64  
 1   title          62423 non-null  object 
 2   genres         62423 non-null  object 
 3   title_cast     17210 non-null  object 
 4   director       17404 non-null  object 
 5   runtime        15189 non-null  float64
 6   budget         7906 non-null   object 
 7   plot_keywords  16200 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 4.3+ MB


In [7]:
# Remove Un-necessary columns which I think are not needed
movies_metadata = movies_metadata.drop(['genres'],
               axis = 1)

In [8]:
movies_metadata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62423 entries, 0 to 62422
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movieId        62423 non-null  int64  
 1   title          62423 non-null  object 
 2   title_cast     17210 non-null  object 
 3   director       17404 non-null  object 
 4   runtime        15189 non-null  float64
 5   budget         7906 non-null   object 
 6   plot_keywords  16200 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 3.8+ MB


# Calculating Avg and Volume of Ratings

In [9]:
ratings_summary = ratings.groupby('movieId') \
       .agg({'rating':'mean', 'userId':'count'}) \
       .rename(columns={'rating':'rating_avg','userId':'rating_count'}) \
       .reset_index()

movie_ratings = pd.merge(ratings_summary, movies)
movie_ratings.head(3)

,movieId,rating_avg,rating_count,title,genres
0,1,3.889971,23062,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,3.263414,9654,Jumanji (1995),Adventure|Children|Fantasy
2,3,3.132325,4727,Grumpier Old Men (1995),Comedy|Romance


# genres to lists

In [10]:
movie_ratings.genres = movie_ratings.genres.str.split('|')
movie_ratings.head(3)

,movieId,rating_avg,rating_count,title,genres
0,1,3.889971,23062,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,3.263414,9654,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,3.132325,4727,Grumpier Old Men (1995),"[Comedy, Romance]"


In [11]:
new_movies_metadata = movies_metadata.drop(['movieId'],axis = 1)
movies_full = pd.merge(movie_ratings, new_movies_metadata , on = ['title'])
movies_full.head()

,movieId,rating_avg,rating_count,title,genres,title_cast,director,runtime,budget,plot_keywords
0,1,3.889971,23062,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,John Lasseter,81.0,"$30,000,000",toy|rivalry|cowboy|cgi animation
1,2,3.263414,9654,Jumanji (1995),"[Adventure, Children, Fantasy]",Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,Jonathan Hensleigh,104.0,"$65,000,000",board game|adventurer|fight|game
2,3,3.132325,4727,Grumpier Old Men (1995),"[Comedy, Romance]",Walter Matthau|Jack Lemmon|Sophia Loren|Ann-Ma...,Mark Steven Johnson,101.0,"$25,000,000",boat|lake|neighbor|rivalry
3,4,2.878099,968,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",Whitney Houston|Angela Bassett|Loretta Devine|...,Terry McMillan,124.0,"$16,000,000",black american|husband wife relationship|betra...
4,5,3.059165,4648,Father of the Bride Part II (1995),[Comedy],Steve Martin|Diane Keaton|Martin Short|Kimberl...,Albert Hackett,106.0,"$30,000,000",fatherhood|doberman|dog|mansion


# Bringing in Keywords and Tags

In [21]:
# Capturing Tags only if its relevance is higher than 80% to a movie
gs = genome_scores[genome_scores['relevance'] > 0.80]
gs.head(5)

,movieId,tagId,relevance
28,1,29,0.89375
62,1,63,0.94725
63,1,64,0.98425
185,1,186,0.95475
192,1,193,0.81450


In [13]:
tags_scores = pd.merge(genome_tags ,gs , on = ['tagId'])
tags_scores.sort_values(['movieId'], ascending=True).head(10)

,tagId,tag,movieId,relevance
84987,717,nostalgic,1,0.87225
21821,204,children,1,0.95975
64820,536,imdb top 250,1,0.93675
23277,215,classic,1,0.81500
60023,497,heartwarming,1,0.81225
57663,469,great movie,1,0.90725
128599,1062,unlikely friendships,1,0.89625
26990,244,computer animation,1,0.99875
126794,1036,toys,1,0.99925
3961,29,adventure,1,0.89375


In [14]:
ky = tags_scores[['tag','movieId']]
ky.drop_duplicates(keep=False,inplace=True) 

A_ky = ky.groupby('tag') \
       .agg({'movieId':'count'}) \
       .rename(columns={'movieId':'count'}) \
       .reset_index()
A_ky.head(5)

,tag,count
0,007,44
1,007 (series),49
2,18th century,100
3,1920s,6
4,1930s,163


# Fuzzy String Matching With Pandas and FuzzyWuzzy


* The idea of using this is for that we want to categorize similar keywords together to identify keywords that can influence a movie rating
* The function tries to compare popular keywords identified by having counts > 100 and replacing the not so popular tags with the most relevant tags 

In [23]:
# Keyword Analysis using Fuzzy Matching
from fuzzywuzzy import process
names_array=[]
ratio_array=[]

def match_names(wrong_names,correct_names):
    for row in wrong_names:
        x=process.extractOne(row, correct_names)
        names_array.append(x[0])
        ratio_array.append(x[1])
    return names_array,ratio_array
 
df= A_ky
wrong_names=df['tag'].dropna().values
 
#Correct tags dataset
choices_df=A_ky[A_ky['count'] > 100]
correct_names=choices_df['tag'].values 
 
name_match,ratio_match=match_names(wrong_names,correct_names)
 
df['correct_tag_name']=pd.Series(name_match)
df['tag_names_ratio']=pd.Series(ratio_match)

In [16]:
output = df[['tag','correct_tag_name','tag_names_ratio' ]]
output.head()
new_tags = output[(output.tag_names_ratio >= 81) & (output.tag_names_ratio <= 100)]
correct_tags = pd.merge(ky[['movieId','tag']],new_tags[['tag','correct_tag_name']], on = 'tag')
tags_clean  = correct_tags.groupby(['correct_tag_name','movieId']) \
                           .agg({'tag':'count'}) \
                           .rename(columns={'tag':'count'}) \
                           .reset_index()
tags_clean  = tags_clean.rename(columns={'correct_tag_name' : 'tag'})
tags_clean.head(12)

,tag,movieId,count
0,1930s,30,1
1,1930s,41,1
2,1930s,197,1
3,1930s,348,1
4,1930s,483,1
5,1930s,513,1
6,1930s,533,1
7,1930s,728,1
8,1930s,753,1
9,1930s,897,1


In [17]:
keywords = pd.merge(movies_full, tags_clean[['tag','movieId']] , on = ['movieId'])
keywords.head()

,movieId,rating_avg,rating_count,title,genres,title_cast,director,runtime,budget,plot_keywords,tag
0,1,3.889971,23062,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,John Lasseter,81.0,"$30,000,000",toy|rivalry|cowboy|cgi animation,adventure
1,1,3.889971,23062,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,John Lasseter,81.0,"$30,000,000",toy|rivalry|cowboy|cgi animation,animated
2,1,3.889971,23062,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,John Lasseter,81.0,"$30,000,000",toy|rivalry|cowboy|cgi animation,animation
3,1,3.889971,23062,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,John Lasseter,81.0,"$30,000,000",toy|rivalry|cowboy|cgi animation,cartoon
4,1,3.889971,23062,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,John Lasseter,81.0,"$30,000,000",toy|rivalry|cowboy|cgi animation,childhood
